In [ ]:
# 参考情報 Cythonの使い方
#https://qiita.com/kenmatsu4/items/7c08a85e41741e95b9ba

#このセルから順番に実行RUNすると
%load_ext Cython

In [2]:
%%cython
import numpy as np

def mandelblot_div(A,B,n_):#発散速度？的なやつ
    xy = np.full((A.shape[0],A.shape[1]),1.0)
    cdef int i,j,k
    cdef double x=0.0
    cdef double y=0.0
    cdef double x_1,y_1
    cdef double ret_
    for i,a in enumerate(A):
        for j,b in enumerate(a):
            x=0.0
            y=0.0
            x_1=x
            y_1=y
            for k in range(n_):
                x_1=x
                y_1=y
                x= x_1**2-y_1**2+A[i,j]
                y= 2*x_1*y_1+B[i,j]
                ret_ = x**2+y**2
                if ret_>4:
                    xy[i,j]=k/n_
                    break
    return xy

cdef double xxyy(double x, double y):
    return x**2+y**2

import numpy as np

def mandelblot3 (A,B,n_):
    cdef int i,j,k
    cdef double x=0.0
    cdef double y=0.0
    cdef double x_1,y_1
    cdef double ret_
    xy = np.full((A.shape[0],A.shape[1]),1.0)
    for i,a in enumerate(A):
        for j,b in enumerate(a):
            x=0
            y=0
            x_1=x
            y_1=y
            for k in range(n_):
                x_1=x
                y_1=y
                x= x_1**2-y_1**2+A[i,j]
                y= 2*x_1*y_1+B[i,j]
                ret_ = x**2+y**2
                if ret_>4:
                    xy[i,j]=k/n_
                    break
    return xy


In [3]:
#マンデルブロ集合の描画　2021.01.08
#動作環境　Windows10 Anaconda JupyterNotebook Python 3.7, OpenCV 4.5
#参考情報
# 環境構築 https://qiita.com/tttamaki/items/0a269b37414340983071
# OpenCV https://whitecat-student.hatenablog.com/entry/2016/11/09/225631

#　Fキー：2値計算と発散速度計算の切り替え
#　Cキー：カラーマップ（色の付け方）の切り替え
#　ESCキー：終了
#　マウスクリック：既定の倍率でクリック座標を中心に拡大

import numpy as np
import cv2 #OpenCV

#定数定義

IsColoer=False# True:カラー描画　False:2値
ColoerMap=2 #cv.COLORMAP_JET
window_name = "Mandelbrot set."#表示するWindow名

#収束判定に用いる次数
N_ = 200

#初期計算の範囲
X_MAX=0.5
X_MIN=-1.5
Y_MAX=1.0
Y_MIN=-1.0

SCALE=10.0

#解像度
RESOLUTION_X=200#実軸ドット数
M_X=RESOLUTION_X
M_Y=round(M_X/(X_MAX-X_MIN)*(Y_MAX-Y_MIN))#虚軸ドット数

_WIDTH = M_X
_HEIGHT = M_Y

C_X=(X_MAX-X_MIN)/2.0+X_MIN
C_Y=(Y_MAX-Y_MIN)/2.0+Y_MIN

POS2=(C_X,C_Y)

w=(X_MAX-X_MIN)*SCALE / 2
h=(Y_MAX-Y_MIN)*SCALE / 2
X_MIN=C_X-w
X_MAX=C_X+w
Y_MIN=C_Y-h
Y_MAX=C_Y+h

def mandelblot (A,B,n_):#うるしおーるのとおなじ関数
    x,y=[0]*2
    for i in range(n_):
        x,y= x**2-y**2+A,2*x*y+B
    return x**2+y**2<4



class mouseParam:#参考情報のURL参照
    def __init__(self, input_img_name):
        #マウス入力用のパラメータ
        self.mouseEvent = {"x":None, "y":None, "event":None, "flags":None}
        #マウス入力の設定
        cv2.setMouseCallback(input_img_name, self.__CallBackFunc, None)
    
    #コールバック関数
    def __CallBackFunc(self, eventType, x, y, flags, userdata):
        
        self.mouseEvent["x"] = x
        self.mouseEvent["y"] = y
        self.mouseEvent["event"] = eventType    
        self.mouseEvent["flags"] = flags    

    #マウス入力用のパラメータを返すための関数
    def getData(self):
        return self.mouseEvent
    
    #マウスイベントを返す関数
    def getEvent(self):
        return self.mouseEvent["event"]                

    #マウスフラグを返す関数
    def getFlags(self):
        return self.mouseEvent["flags"]                

    #xの座標を返す関数
    def getX(self):
        return self.mouseEvent["x"]  

    #yの座標を返す関数
    def getY(self):
        return self.mouseEvent["y"]  

    #xとyの座標を返す関数
    def getPos(self):
        return (self.mouseEvent["x"], self.mouseEvent["y"])

    
def xyToReal (x,y):
    X=(x/M_X)*(X_MAX-X_MIN)+X_MIN
    Y=(y/M_Y)*(Y_MAX-Y_MIN)+Y_MIN
    return (X,Y)

def update_scale(x,y):
    global X_MAX,X_MIN,Y_MAX,Y_MIN,SCALE
    w=(X_MAX-X_MIN)/SCALE / 2
    h=(Y_MAX-Y_MIN)/SCALE / 2
    X_MIN=x-w
    X_MAX=x+w
    Y_MIN=y-h
    Y_MAX=y+h

def update(x,y):
    global X_MAX,X_MIN,Y_MAX,Y_MIN,SCALE
    
    a=np.linspace(X_MIN,X_MAX,M_X)
    b=np.linspace(Y_MIN,Y_MAX,M_Y)
    A,B=np.meshgrid(a,b)
    if IsColoer==True:
        return np.uint8(255*mandelblot_div(A,B,N_))
    else:
        return np.uint8(255*mandelblot(A,B,N_))
    
#ウィンドウ表示および更新
def refresh():
    global window_name,POS2,ColoerMap
    print("(x,y) =",POS2[0],POS2[1])#表示のセンター座標
    ret = update(POS2[0],POS2[1])#マンデルブロ集合の計算
    im_color=cv2.applyColorMap(ret, ColoerMap)#グレースケールをカラーマップして表示
    cv2.imshow(window_name, im_color)
    
def main_():
    global window_name,POS2,IsColoer,ColoerMap
    print('OpenCV:'+cv2.__version__)
    
    update_scale(C_X,C_Y)
    refresh()
    
    #コールバックの設定
    mouseData = mouseParam(window_name)
    
    print('Hit ESC key to close this window.')
    while 1:
        #左クリックの座標を中心にSCALE倍して表示を更新
        if mouseData.getEvent() == cv2.EVENT_LBUTTONDOWN:
            pos=mouseData.getPos()
            POS2=xyToReal(pos[0],pos[1])
            update_scale(POS2[0],POS2[1])
            refresh()
        
        key = cv2.waitKey(100)#キー入力待ち 100ms
        #print(key)
        if key == 102:#F キー入力で計算方法切り替え
            IsColoer = not IsColoer
            %time refresh()
            
        if key == 99:#C カラーマップ切り替え
            ColoerMap=(ColoerMap+1)%21
            print("ColoerMap =",ColoerMap)
            refresh()
            
        if key == 27:#ESCキー入力があったら終了
            cv2.destroyAllWindows()
            break;
            
    cv2.destroyAllWindows()    
    print("Quit.")
    
if __name__ == "__main__":
    main_()

OpenCV:4.5.0
(x,y) = -0.5 0.0
Hit ESC key to close this window.


C:\Users\himukai_takumi\.conda\envs\py37_cv45\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in square
C:\Users\himukai_takumi\.conda\envs\py37_cv45\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in subtract
C:\Users\himukai_takumi\.conda\envs\py37_cv45\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in multiply
C:\Users\himukai_takumi\.conda\envs\py37_cv45\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in square


(x,y) = -0.78 -0.14
(x,y) = -0.769 -0.10000000000000003
(x,y) = -0.7655 -0.09810000000000003
(x,y) = -0.76527 -0.09761000000000003
(x,y) = -0.765299 -0.09754300000000003


C:\Users\himukai_takumi\.conda\envs\py37_cv45\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in add


(x,y) = -0.7653009 -0.09754210000000003
(x,y) = -0.7653003199999999 -0.09754256000000003
(x,y) = -0.7653002899999999 -0.09754253400000003
(x,y) = -0.7653002882999999 -0.09754253410000002
(x,y) = -0.7653002881599998 -0.09754253417000003
(x,y) = -0.7653002881179999 -0.09754253418300003
(x,y) = -0.7653002881212999 -0.09754253418130003
(x,y) = -0.7653002881212899 -0.09754253418075003
Quit.
